Conexión por SSH
==

In [5]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "gonzalo"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

AuthenticationException: Authentication failed.

Ejecución de comandos
==

Implementé un *servidor MCLP*, el cual escucha instrucciones para generar bins.

El siguiente código lanza el servidor en el background, carga una instancia y lo deja escuchando peticiones por el puerto 8080

In [26]:
transport = ssh.get_transport()
channel = transport.open_session()
channel.exec_command("./mclp/Metasolver/BASIC_BSG_MCLP mclp/Metasolver/problems/mclp/benchs/class1/100.txt -i 2")

**Generación de conjunto inicial de bins con BSG**

La siguiente función genera la lista inicial de bins. Cada línea representa un bin indicando el porcentaje de llenado y los ids de sus cajas.

Es necesario modificarla para que en vez de imprimir el contenido de los bins, retorne una estructura adecuada.

In [1]:
def generate_bins():
    stdin, stdout, stderr = ssh.exec_command("echo generate_bins | netcat localhost 8080")
    flag = False
    for line in stdout.readlines():
        if "BINS:" in line: 
            flag = True
            continue
        if flag:
            print(line.strip())
generate_bins()

NameError: name 'ssh' is not defined

**Construcción de bin priorizando cajas (BSG)**

In [ ]:
def generate_bin(boxes, priority_boxes):
    boxes_str = [str(box) for box in boxes]
    pboxes_str = [str(pbox) for pbox in priority_boxes]    
    stdin, stdout, stderr = ssh.exec_command("echo generate_bin " + " ".join(boxes_str) + " -1 " + " ".join(pboxes_str) +  " -2 | netcat localhost 8080")
    for line in stdout.readlines(): print(line.strip())
        
generate_bin([0,1,8,53,45],[45])

Para cerrar el servidor MCLP

In [ ]:
def close_mclp_server():
    stdin, stdout, stderr = ssh.exec_command("echo END | netcat localhost 8080")
    print(stdout.readlines()[0])
    
close_mclp_server()

Faltaría:

* Cargar la instancia en python (cajas con sus dimensiones)
* Implementar algoritmo.

A grandes rasgos el algoritmo hace los siguiente:
1. Generación de bins iniciales usando **BSG**
2. Selección de bin a desarmar y almacenar cajas en $C$
3. Mientras $C$ no quede vacío o máximo de iteraciones:

   1. Seleccionar caja $c$ de $C$
   2. Seleccionar bin de destino $B$
   3. Usar **BSG** para generar bin $B'$ usando cajas $B \cup \{c\}$, priorizando $c$. Es posible que **BSG** retorne conjunto de cajas residuales $R$
   4. Si $R$ es mejor que $c$, $B$ se reemplaza por $B'$ en el conjunto de bins y $C \gets C \cup R$
   
 4. Volver a 2 (seleccionar otro bin para desarmar)

In [6]:
def solve(solver, s0, swapIter):
    cantBoxes = s0.nbLeftBoxes
    maxVolBin = s0.getContVol
    boxes = set()
    bestBins = None
    bins = None

    for box in s0.boxes:
        boxes.add(box)
    
    #Llamada a BSG
    bins = generador()
    
    firstSol = len(bins)
    for i in range(swapIter):
        swapBoxes(bins, maxVolBin)
    
    return len(bins)


In [ ]:
def swapBoxes(bins, maxVolVin):
    breakedBin = bins.pop()
    
    while(len(breakedBin) == 0 or iter <100):
        pivBin = bins.pop()
        priorityBox = set()

        complexBox = getComplexBox(breakedBin)
        priorityBox.add(complexBox)
        pivBin.add(complexBox)

        #Metodo de llamada al BSG
        boxes = generador()

        #Se verifica la si el cambio es coherente o no
        if(accordChange(pivBin, boxes[0], priorityBox, boxes[1])):
            #Se inserta el nuevo bin dentro del conjunto de bins
            bins.add(boxes[0])
            #Se elimina (desde el conjunto de cajas a repartir) la caja insertada en el nuevo bin 
            breakedBin.remove(complexBox)
           
            #Se agregan las cajas que se cambiaron
            if(len(boxes[1]) > 0 ):
                for b in boxes[1]:
                    breakedBin.add(b)

        else:
            pivBin.remove(complexBox)
            bins.append(pivBin)

    x = generador()
    for b in x:
        bins.append(b)
        

In [ ]:
def calculateComplex(idBox, dim):
    x = dim[0]
    y = dim[1]
    z = dim[2]
    f1 = 0
    f2 = 0
    f3 = 0
    expo = 1
    vol = x*y*z

    if( x > y and x != 0 ):
        f1 = y/x
    elif( y != 0 ):
        f1 = x/y
    
    if( x > z and x != 0):
        f2 = z/x
    elif( z != 0):
        f2 = x/z

    if( z > y and z != 0):
        f3 = y/z
    elif ( z != 0 ):
        f3 = y/z
    return ((f1*f2*f3)**expo) * vol

In [ ]:
def acumulateComplex(bin):
    aux = 0
    boxShape = None
    for b in bin:
        boxShape = None #Cargar instancia
        aux += calculateComplex(v,boxShape)
    return aux

In [ ]:
def getComplexBox(bins):
    idBox = None
    eval = 0
    for b in bins:
        boxShape = None #Cargar instancia
        currentEval = calculateComplex(b,boxShape)    
        if (idBox is None):
            idBox = b
            eval = currentEval
        else:
            if(currentEval > eval):
                idBox = b
                eval = currentEval

    return idBox

In [ ]:
import random as rand

def popRandomBin(bins):
    rPos = rand.random()%len(bins)
    theBin = bins.pop(rPos)
    return theBin